# <center>Welcome FSP solved with local search based metaheuristics</center>

This notebook presents a practical approach to solving the flowshop problem by implementing well-known local search based metaheuristics. These metaheuristics are effective in generating high-quality solutions for large instances of the problem, requiring only a reasonable amount of computational resources. Compared to heuristics, local search metaheuristics are more effective for solving the flowshop problem because they can escape from local optima and find better solutions

# Table of Contents

1. [Data utils](#Data-utils)
2. [Neighborhood generation](#Neighborhood-generation)
2. 1. [SWAP](#SWAP)
4. [Random SWAP](#Random-SWAP)
5. [Best SWAP](#Best-SWAP)
6. [First Admissible SWAP](#First-Admissible-SWAP)
7. 4. [Random SWAP](#Random-SWAP)

3. [Local Based Metaheuristics](#Local-Based-Metaheuristics)
4. [Tests](#Tests)
5. [Palmer Heuristic](#palmer-heuristic)
6. [PRSKE Heuristic](#prske-heuristic)
8. [Chen Heuristic](#Chen-heuristic-(1983))
7. [Weighted CDS Heuristic](#Weighted-CDS-heuristic)
9. [Gupta Heuristic](#gupta-heuristic)
10. [NRH Heuristic (NEW Ramzi Heuristic)](#nrh-heuristic-new-ramzi-heuristic)
11. [Kusiak Heuristic ](#Kusiak-heuristic)
12. [ Comparison Figure   ](#Comparison-Figure)




## Data utils

In [62]:
import numpy as np
import random
import time
import math
import pandas as pd
import matplotlib.pyplot as plt
from utils.benchmarks import benchmarks, upper_bound

### Path Cost calculation function :
Used to calculate the cost of current node, which is the correct cost starting for the actual path of executed jobs

In [3]:
def calculate_makespan(processing_times, sequence):
    n_jobs = len(sequence)
    n_machines = len(processing_times[0])
    end_time = [[0] * (n_machines + 1) for _ in range(n_jobs + 1)]

    for j in range(1, n_jobs + 1):
        for m in range(1, n_machines + 1):
            end_time[j][m] = max(end_time[j][m - 1], end_time[j - 1]
                                 [m]) + processing_times[sequence[j - 1]][m - 1]

    return end_time[n_jobs][n_machines]

### Random data for tests

In [8]:
rnd_data = np.random.randint(size=(20,5), low=5, high=120)
permutation = np.random.permutation(20).tolist()
print(rnd_data, "\n")
print('Initial solution:', permutation, "\n")

Cmax = calculate_makespan(rnd_data, permutation)
print(f'Makespan: {Cmax}')

[[  8  55  62  22  39]
 [ 79  62  65  12 113]
 [ 86  96  59 108 110]
 [ 35  12  56  65  99]
 [ 99  55  80 116   5]
 [ 53 116   5  31   9]
 [ 52  16  64 101  99]
 [ 73  26  45 113 119]
 [ 88  74  94 109  51]
 [106  48   7 112  96]
 [108 104  66  84  32]
 [ 70  43 106  13  76]
 [ 22 113 117  20 116]
 [ 82 113  39  74  98]
 [ 46 113  37   9  50]
 [ 62  15  97 117 100]
 [ 83 113  81  55  29]
 [ 87   6  91  54 106]
 [109   8  71  98   8]
 [ 25 109 101  80  88]] 

Initial solution: [16, 14, 0, 19, 10, 2, 13, 3, 5, 7, 4, 9, 6, 12, 18, 1, 8, 15, 17, 11] 

Makespan: 2127


### Gantt graph generator

In [3]:
def generate_gantt_chart(processing_times, seq, interval=50, labeled=True):
    data = processing_times.T
    nb_jobs, nb_machines = processing_times.shape
    schedules = np.zeros((nb_machines, nb_jobs), dtype=dict)
    # schedule first job alone first
    task = {"name": "job_{}".format(
        seq[0]+1), "start_time": 0, "end_time": data[0][seq[0]]}

    schedules[0][0] = task
    for m_id in range(1, nb_machines):
        start_t = schedules[m_id-1][0]["end_time"]
        end_t = start_t + data[m_id][0]
        task = {"name": "job_{}".format(
            seq[0]+1), "start_time": start_t, "end_time": end_t}
        schedules[m_id][0] = task

    for index, job_id in enumerate(seq[1::]):
        start_t = schedules[0][index]["end_time"]
        end_t = start_t + data[0][job_id]
        task = {"name": "job_{}".format(
            job_id+1), "start_time": start_t, "end_time": end_t}
        schedules[0][index+1] = task
        for m_id in range(1, nb_machines):
            start_t = max(schedules[m_id][index]["end_time"],
                          schedules[m_id-1][index+1]["end_time"])
            end_t = start_t + data[m_id][job_id]
            task = {"name": "job_{}".format(
                job_id+1), "start_time": start_t, "end_time": end_t}
            schedules[m_id][index+1] = task

    # create a new figure
    fig, ax = plt.subplots(figsize=(18, 8))

    # set y-axis ticks and labels
    y_ticks = list(range(len(schedules)))
    y_labels = [f'Machine {i+1}' for i in y_ticks]
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(y_labels)

    # calculate the total time
    total_time = max([job['end_time'] for proc in schedules for job in proc])

    # set x-axis limits and ticks
    ax.set_xlim(0, total_time)
    x_ticks = list(range(0, total_time+1, interval))
    ax.set_xticks(x_ticks)

    # set grid lines
    ax.grid(True, axis='x', linestyle='--')

    # create a color dictionary to map each job to a color
    color_dict = {}
    for proc in schedules:
        for job in proc:
            if job['name'] not in color_dict:
                color_dict[job['name']] = (np.random.uniform(
                    0, 1), np.random.uniform(0, 1), np.random.uniform(0, 1))

    # plot the bars for each job on each processor
    for i, proc in enumerate(schedules):
        for job in proc:
            start = job['start_time']
            end = job['end_time']
            duration = end - start
            color = color_dict[job['name']]
            ax.barh(i, duration, left=start, height=0.5,
                    align='center', color=color, alpha=0.8)
            if labeled:
                # add job labels
                label_x = start + duration/2
                label_y = i
                ax.text(
                    label_x, label_y, job['name'][4:], ha='center', va='center', fontsize=10)

    plt.show()

### INITIAL SOLUTION METHODS

In [41]:
from utils.initial_sol_methods import neh_algorithm
from utils.initial_sol_methods import PRSKE
from utils.initial_sol_methods import ham_heuristic

## Neighborhood generation

### SWAP

In [9]:
def swap(solution, i, k):
    sol = solution.copy()
    sol[i], sol[k] = sol[k], sol[i]
    return sol

### Random SWAP

In [10]:
def random_swap(solution, processing_times):
    i = random.choice(list(solution))
    j = random.choice(list(solution))

    while i == j:
        j = random.choice(list(solution))

    new_solution = swap(solution, i, j)
    
    return new_solution, calculate_makespan(processing_times, new_solution)

In [26]:
neighbor_sol, neighbor_cmax = random_swap(permutation, rnd_data)

print("Original_solution: ", permutation)
print("Cmax = ", Cmax)

print("\nNeighbor_solution: ", neighbor_sol)
print("Neighor_cmax = ", neighbor_cmax)

Original_solution:  [16, 14, 0, 19, 10, 2, 13, 3, 5, 7, 4, 9, 6, 12, 18, 1, 8, 15, 17, 11]
Cmax =  2127

Neighbor_solution:  [4, 14, 0, 19, 10, 2, 13, 3, 5, 7, 16, 9, 6, 12, 18, 1, 8, 15, 17, 11]
Neighor_cmax =  2043


### Best SWAP

In [14]:
def best_swap(solution, processing_times):
    sequence = solution.copy()
    num_jobs = len(solution)
    Cmax = calculate_makespan(processing_times, solution)

    for i in range(num_jobs):
        for j in range(i+1, num_jobs):
            new_solution = swap(sequence, i, j)
            makespan = calculate_makespan(processing_times, new_solution)

            if makespan < Cmax:
                sequence = new_solution
                Cmax = makespan

    return sequence, Cmax

In [25]:
neighbor_sol, neighbor_cmax = best_swap(permutation, rnd_data)

print("Original_solution: ", permutation)
print("Cmax = ", Cmax)

print("\nNeighbor_solution: ", neighbor_sol)
print("Neighor_cmax = ", neighbor_cmax)

Original_solution:  [16, 14, 0, 19, 10, 2, 13, 3, 5, 7, 4, 9, 6, 12, 18, 1, 8, 15, 17, 11]
Cmax =  2127

Neighbor_solution:  [15, 12, 7, 19, 6, 1, 17, 13, 3, 2, 11, 9, 10, 4, 16, 18, 8, 0, 5, 14]
Neighor_cmax =  1734


### First Admissible SWAP

In [17]:
def first_admissible_swap(solution, processing_times):

    num_jobs = len(solution)
    Cmax = calculate_makespan(processing_times, solution)

    for i in range(num_jobs):
        for j in range(i+1, num_jobs):
            new_solution = swap(solution, i, j)
            makespan = calculate_makespan(processing_times, new_solution)

            if makespan < Cmax:
                return new_solution, makespan

    return solution, Cmax 

In [27]:
neighbor_sol, neighbor_cmax = first_admissible_swap(permutation, rnd_data)

print("Original_solution: ", permutation)
print("Cmax = ", Cmax)

print("\nNeighbor_solution: ", neighbor_sol)
print("Neighor_cmax = ", neighbor_cmax)

Original_solution:  [16, 14, 0, 19, 10, 2, 13, 3, 5, 7, 4, 9, 6, 12, 18, 1, 8, 15, 17, 11]
Cmax =  2127

Neighbor_solution:  [14, 16, 0, 19, 10, 2, 13, 3, 5, 7, 4, 9, 6, 12, 18, 1, 8, 15, 17, 11]
Neighor_cmax =  2090


### First and Best Admissible SWAP

In [20]:
def fba_swap(solution, processing_times, best_global_sol):
    sequence = solution.copy()
    num_jobs = len(sequence)
    Cmax = calculate_makespan(processing_times, sequence)
    Smax = calculate_makespan(processing_times, best_global_sol)
    for i in range(num_jobs):
        for j in range(i+1, num_jobs):
            new_solution = swap(solution, i, j)
            makespan = calculate_makespan(processing_times, new_solution)

            # First improving solution
            if makespan < Cmax:
                # Improves the global solution
                if makespan < Smax:
                    return new_solution, makespan, new_solution 
                Cmax = makespan
                sequence = new_solution                

    return sequence, Cmax, best_global_sol 

In [29]:
best_global = np.random.permutation(20).tolist()

while calculate_makespan(rnd_data, best_global) > Cmax:
    best_global = np.random.permutation(20).tolist()

neighbor_sol, neighbor_cmax, best_global_found = fba_swap(permutation, rnd_data, best_global)

print("Original_solution: ", permutation)
print("Cmax = ", Cmax)

print("\nBest_original_global_solution: ", best_global)
print("Best_global_initial_cmax = ", calculate_makespan(rnd_data, best_global))

print("\nNeighbor_solution: ", neighbor_sol)
print("Neighor_cmax = ", neighbor_cmax)

print("\nBest_global_solution: ", best_global_found)
print("Best_global_cmax = ", calculate_makespan(rnd_data, best_global_found))

Original_solution:  [16, 14, 0, 19, 10, 2, 13, 3, 5, 7, 4, 9, 6, 12, 18, 1, 8, 15, 17, 11]
Cmax =  2127

Best_original_global_solution:  [10, 3, 19, 6, 15, 14, 1, 5, 9, 0, 12, 8, 17, 18, 13, 7, 2, 16, 4, 11]
Best_global_initial_cmax =  1898

Neighbor_solution:  [15, 14, 0, 19, 10, 2, 13, 3, 5, 7, 4, 9, 6, 12, 18, 1, 8, 16, 17, 11]
Neighor_cmax =  1968

Best_global_solution:  [10, 3, 19, 6, 15, 14, 1, 5, 9, 0, 12, 8, 17, 18, 13, 7, 2, 16, 4, 11]
Best_global_cmax =  1898


## Local Based Metaheuristics

### Simulated annealing (RS) 

#### General method to get neighbors (except first and best admissible)

In [37]:
def get_neighbor(processing_times, solution, method='random_swap'):
    if method == 'random_swap':
        sol, val = random_swap(solution, processing_times)
    elif method == 'best_swap':
        sol, val = best_swap(solution, processing_times)
    elif method == 'first_admissible_swap':
        sol, val = first_admissible_swap(solution, processing_times)
    else:
        i = random.randint(0, 2)
        if i == 0:
            sol, val = random_swap(solution, processing_times)
        elif i == 1:
            sol, val = best_swap(solution, processing_times)
        elif i == 2:
            sol, val = first_admissible_swap(solution, processing_times)
    return sol, val

In [101]:
def RS(processing_times, initial_solution, temp, method='random_swap', alpha=0.6, nb_palier= 10, it_max=100):
    solution = initial_solution.copy()
    makespan = calculate_makespan(processing_times, solution)
    print('init_sol: ',solution, ' makespan = ', makespan, "\n")
    it = 0
    while it < it_max:
        for i in range(nb_palier):
            sol, value = get_neighbor(processing_times, solution, method)
            print('Swap_sol: ',sol,' makespan = ', value)
            delta = makespan - value
            if delta > 0:
                solution = sol
                makespan = value
            else:
                if random.uniform(0, 1) < math.exp(delta / temp):
                    solution = sol
        temp = alpha * temp
        it += 1
    
    return solution

### RS with first and best admissible swap 

In [118]:
def RS_fba(processing_times, initial_solution, intitial_global, temp, alpha=0.6, nb_palier= 1, it_max=100):
    solution = initial_solution.copy()
    makespan = calculate_makespan(processing_times, solution)
    print('init_sol: ',solution, ' makespan = ', makespan)
    it = 0
    print('initial_global_solution: ',intitial_global, ' global_makespan = ', calculate_makespan(processing_times, intitial_global), "\n")
    best_global = intitial_global.copy()
    while it < it_max :
        for i in range(nb_palier):
            sol, value, best_global = fba_swap(solution, processing_times, best_global)
            print('FBA_swap_sol: ',sol,' makespan = ', value)
            delta = makespan - value
            if delta > 0:
                solution = sol
                makespan = value
            else:
                if random.uniform(0, 1) < math.exp(delta / temp):
                    solution = sol
        temp = alpha * temp
        it += 1
    
    return solution, best_global

## Tests

We can adjust the swapping method to see the differences in tests

In [115]:
benchmark = benchmarks[0]

best_global = np.random.permutation(20).tolist()
while calculate_makespan(rnd_data, best_global) > Cmax:
    best_global = np.random.permutation(20).tolist()

### Random initial solution

#### Without FBA swapping method

In [116]:
rs_solution = RS(benchmark, permutation, 5, method='random_swap')

init_sol:  [16, 14, 0, 19, 10, 2, 13, 3, 5, 7, 4, 9, 6, 12, 18, 1, 8, 15, 17, 11]  makespan =  1489 

Swap_sol:  [16, 14, 0, 11, 10, 2, 13, 3, 5, 7, 4, 9, 6, 12, 18, 1, 8, 15, 17, 19]  makespan =  1442
Swap_sol:  [16, 14, 0, 18, 10, 2, 13, 3, 5, 7, 4, 9, 6, 12, 11, 1, 8, 15, 17, 19]  makespan =  1343
Swap_sol:  [16, 14, 0, 18, 10, 4, 13, 3, 5, 7, 2, 9, 6, 12, 11, 1, 8, 15, 17, 19]  makespan =  1387
Swap_sol:  [16, 14, 0, 18, 10, 2, 13, 3, 5, 7, 4, 1, 6, 12, 11, 9, 8, 15, 17, 19]  makespan =  1348
Swap_sol:  [16, 14, 0, 18, 10, 2, 13, 3, 5, 7, 15, 9, 6, 12, 11, 1, 8, 4, 17, 19]  makespan =  1345
Swap_sol:  [16, 14, 0, 18, 10, 7, 13, 3, 5, 2, 4, 9, 6, 12, 11, 1, 8, 15, 17, 19]  makespan =  1360
Swap_sol:  [18, 14, 0, 16, 10, 2, 13, 3, 5, 7, 4, 9, 6, 12, 11, 1, 8, 15, 17, 19]  makespan =  1343
Swap_sol:  [18, 14, 0, 16, 15, 2, 13, 3, 5, 7, 4, 9, 6, 12, 11, 1, 8, 10, 17, 19]  makespan =  1356
Swap_sol:  [18, 14, 0, 16, 10, 17, 13, 3, 5, 7, 4, 9, 6, 12, 11, 1, 8, 15, 2, 19]  makespan =  147

#### With FBA swapping method

In [119]:
rs_fba_solution, best_global_found = RS_fba(benchmark, permutation, best_global, 5)

init_sol:  [16, 14, 0, 19, 10, 2, 13, 3, 5, 7, 4, 9, 6, 12, 18, 1, 8, 15, 17, 11]  makespan =  1489
initial_global_solution:  [5, 14, 16, 4, 19, 1, 9, 2, 0, 7, 10, 15, 3, 12, 8, 13, 6, 18, 17, 11]  global_makespan =  1482 

FBA_swap_sol:  [5, 14, 0, 19, 10, 2, 13, 3, 16, 7, 4, 9, 6, 12, 18, 1, 8, 15, 17, 11]  makespan =  1451
FBA_swap_sol:  [14, 5, 0, 19, 10, 2, 13, 3, 16, 7, 4, 9, 6, 12, 18, 1, 8, 15, 17, 11]  makespan =  1444
FBA_swap_sol:  [12, 5, 0, 19, 10, 2, 13, 3, 16, 7, 4, 9, 6, 14, 18, 1, 8, 15, 17, 11]  makespan =  1442
FBA_swap_sol:  [8, 5, 0, 19, 10, 2, 13, 3, 16, 7, 4, 9, 6, 14, 18, 1, 12, 15, 17, 11]  makespan =  1438
FBA_swap_sol:  [13, 5, 0, 19, 10, 2, 8, 3, 16, 7, 4, 9, 6, 14, 18, 1, 12, 15, 17, 11]  makespan =  1434
FBA_swap_sol:  [3, 5, 0, 19, 10, 2, 8, 13, 16, 7, 4, 9, 6, 14, 18, 1, 12, 15, 17, 11]  makespan =  1433
FBA_swap_sol:  [5, 3, 0, 19, 10, 2, 8, 13, 16, 7, 4, 9, 6, 14, 18, 1, 12, 15, 17, 11]  makespan =  1432
FBA_swap_sol:  [14, 3, 0, 19, 10, 2, 8, 13, 16, 

In [120]:
print('Results of random:')
print(f'First sequence: {permutation} with a makespan of {calculate_makespan(benchmark, permutation)}.')
print('\nResults of RS:')
print(f'Best solution: {rs_solution} with a makespan of {calculate_makespan(benchmark, rs_solution)}.')
print('\nResults of RS_FBA:')
print(f'Best solution: {rs_fba_solution} with a makespan of {calculate_makespan(benchmark, rs_fba_solution)}.')
print(f'Best global solution: {best_global_found} with a makespan of {calculate_makespan(benchmark, best_global_found)}.')

Results of random:
First sequence: [16, 14, 0, 19, 10, 2, 13, 3, 5, 7, 4, 9, 6, 12, 18, 1, 8, 15, 17, 11] with a makespan of 1489.

Results of RS:
Best solution: [8, 16, 14, 2, 3, 1, 13, 4, 12, 6, 10, 7, 0, 18, 15, 5, 17, 11, 9, 19] with a makespan of 1297.

Results of RS_FBA:
Best solution: [14, 5, 8, 6, 10, 12, 3, 1, 16, 7, 4, 17, 11, 0, 18, 2, 13, 15, 9, 19] with a makespan of 1297.
Best global solution: [14, 5, 8, 6, 10, 12, 3, 1, 16, 7, 4, 17, 11, 0, 18, 2, 13, 15, 9, 19] with a makespan of 1297.


### NEH initial solution

#### Without FBA swapping method

In [106]:
initialSolution, makespan = neh_algorithm(benchmark)
rs_solution = RS(benchmark, initialSolution, 5)

init_sol:  [8, 6, 15, 10, 7, 1, 16, 2, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 19]  makespan =  1334 

Swap_sol:  [8, 6, 15, 10, 7, 1, 19, 2, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 16]  makespan =  1503
Swap_sol:  [8, 6, 1, 10, 7, 15, 16, 2, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 19]  makespan =  1375
Swap_sol:  [8, 6, 15, 10, 7, 1, 16, 2, 14, 13, 17, 3, 9, 4, 0, 18, 5, 11, 12, 19]  makespan =  1369
Swap_sol:  [8, 6, 15, 10, 7, 1, 19, 2, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 16]  makespan =  1503
Swap_sol:  [8, 6, 15, 10, 7, 2, 16, 1, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 19]  makespan =  1334
Swap_sol:  [8, 6, 4, 10, 7, 2, 16, 1, 14, 13, 17, 3, 9, 11, 0, 18, 5, 15, 12, 19]  makespan =  1400
Swap_sol:  [8, 6, 15, 10, 1, 2, 16, 7, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 19]  makespan =  1375
Swap_sol:  [7, 6, 15, 10, 8, 2, 16, 1, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 19]  makespan =  1341
Swap_sol:  [7, 6, 15, 18, 8, 2, 16, 1, 14, 13, 17, 3, 9, 11, 0, 10, 5, 4, 12, 19]  makespan =  143

#### With FBA swapping method

In [121]:
rs_fba_solution, best_global_found = RS_fba(benchmark, initialSolution, best_global, 5)

init_sol:  [3, 17, 10, 1, 9, 11, 6, 4, 19, 15, 18, 5, 0, 12, 8, 13, 14, 7, 16, 2]  makespan =  1593
initial_global_solution:  [5, 14, 16, 4, 19, 1, 9, 2, 0, 7, 10, 15, 3, 12, 8, 13, 6, 18, 17, 11]  global_makespan =  1482 

FBA_swap_sol:  [3, 17, 10, 12, 9, 11, 6, 4, 19, 15, 18, 5, 0, 1, 8, 13, 14, 7, 16, 2]  makespan =  1505
FBA_swap_sol:  [3, 14, 10, 12, 9, 11, 6, 4, 19, 15, 18, 5, 0, 1, 8, 13, 17, 7, 16, 2]  makespan =  1465
FBA_swap_sol:  [3, 14, 10, 16, 9, 11, 6, 4, 19, 15, 18, 5, 0, 1, 8, 13, 17, 7, 12, 2]  makespan =  1441
FBA_swap_sol:  [3, 14, 10, 16, 17, 11, 6, 4, 19, 15, 18, 5, 0, 1, 8, 13, 9, 7, 12, 2]  makespan =  1432
FBA_swap_sol:  [6, 14, 10, 16, 17, 11, 3, 4, 19, 15, 18, 5, 0, 1, 8, 13, 9, 7, 12, 2]  makespan =  1428
FBA_swap_sol:  [13, 14, 10, 16, 17, 11, 3, 4, 19, 15, 18, 5, 0, 1, 8, 6, 9, 7, 12, 2]  makespan =  1424
FBA_swap_sol:  [13, 14, 10, 16, 7, 11, 3, 4, 19, 15, 18, 5, 0, 1, 8, 6, 9, 17, 12, 2]  makespan =  1420
FBA_swap_sol:  [13, 14, 10, 16, 7, 17, 3, 4, 19,

In [122]:
print('Results of NEH:')
print(f'First sequence: {initialSolution} with a makespan of {makespan}.')
print('\nResults of RS:')
print(f'Best solution: {rs_solution} with a makespan of {calculate_makespan(benchmark, rs_solution)}.')
print('\nResults of RS_FBA:')
print(f'Best solution: {rs_fba_solution} with a makespan of {calculate_makespan(benchmark, rs_fba_solution)}.')
print(f'Best global solution: {best_global_found} with a makespan of {calculate_makespan(benchmark, best_global_found)}.')

Results of NEH:
First sequence: [3, 17, 10, 1, 9, 11, 6, 4, 19, 15, 18, 5, 0, 12, 8, 13, 14, 7, 16, 2] with a makespan of 1593.

Results of RS:
Best solution: [8, 16, 14, 2, 3, 1, 13, 4, 12, 6, 10, 7, 0, 18, 15, 5, 17, 11, 9, 19] with a makespan of 1297.

Results of RS_FBA:
Best solution: [12, 14, 10, 16, 13, 17, 3, 4, 19, 15, 18, 5, 0, 1, 8, 9, 6, 11, 7, 2] with a makespan of 1343.
Best global solution: [12, 14, 10, 16, 13, 17, 3, 4, 19, 15, 18, 5, 0, 1, 8, 9, 6, 11, 7, 2] with a makespan of 1343.


### PRSKE initial solution

#### Without FBA swapping method

In [108]:
initialSolution, makespan  = PRSKE(benchmark)
rs_solution = RS(benchmark, initialSolution, 5)

init_sol:  [3, 17, 10, 1, 9, 11, 6, 4, 19, 15, 18, 5, 0, 12, 8, 13, 14, 7, 16, 2]  makespan =  1593 

Swap_sol:  [3, 17, 10, 1, 9, 11, 6, 7, 19, 15, 18, 5, 0, 12, 8, 13, 14, 4, 16, 2]  makespan =  1606
Swap_sol:  [3, 17, 10, 4, 9, 11, 6, 1, 19, 15, 18, 5, 0, 12, 8, 13, 14, 7, 16, 2]  makespan =  1593
Swap_sol:  [3, 17, 10, 4, 9, 11, 6, 1, 19, 15, 18, 5, 0, 16, 8, 13, 14, 7, 12, 2]  makespan =  1522
Swap_sol:  [3, 17, 10, 4, 9, 11, 12, 1, 19, 15, 18, 5, 0, 16, 8, 13, 14, 7, 6, 2]  makespan =  1542
Swap_sol:  [3, 17, 5, 4, 9, 11, 6, 1, 19, 15, 18, 10, 0, 16, 8, 13, 14, 7, 12, 2]  makespan =  1494
Swap_sol:  [3, 17, 5, 4, 9, 15, 6, 1, 19, 11, 18, 10, 0, 16, 8, 13, 14, 7, 12, 2]  makespan =  1502
Swap_sol:  [3, 17, 5, 4, 9, 11, 6, 1, 19, 15, 18, 16, 0, 10, 8, 13, 14, 7, 12, 2]  makespan =  1498
Swap_sol:  [3, 17, 8, 4, 9, 11, 6, 1, 19, 15, 18, 16, 0, 10, 5, 13, 14, 7, 12, 2]  makespan =  1506
Swap_sol:  [3, 17, 5, 4, 9, 11, 6, 19, 1, 15, 18, 16, 0, 10, 8, 13, 14, 7, 12, 2]  makespan =  152

#### With FBA swapping method

In [123]:
rs_fba_solution, best_global_found = RS_fba(benchmark, initialSolution, best_global, 5)

init_sol:  [3, 17, 10, 1, 9, 11, 6, 4, 19, 15, 18, 5, 0, 12, 8, 13, 14, 7, 16, 2]  makespan =  1593
initial_global_solution:  [5, 14, 16, 4, 19, 1, 9, 2, 0, 7, 10, 15, 3, 12, 8, 13, 6, 18, 17, 11]  global_makespan =  1482 

FBA_swap_sol:  [3, 17, 10, 12, 9, 11, 6, 4, 19, 15, 18, 5, 0, 1, 8, 13, 14, 7, 16, 2]  makespan =  1505
FBA_swap_sol:  [3, 14, 10, 12, 9, 11, 6, 4, 19, 15, 18, 5, 0, 1, 8, 13, 17, 7, 16, 2]  makespan =  1465
FBA_swap_sol:  [3, 14, 10, 16, 9, 11, 6, 4, 19, 15, 18, 5, 0, 1, 8, 13, 17, 7, 12, 2]  makespan =  1441
FBA_swap_sol:  [3, 14, 10, 16, 17, 11, 6, 4, 19, 15, 18, 5, 0, 1, 8, 13, 9, 7, 12, 2]  makespan =  1432
FBA_swap_sol:  [6, 14, 10, 16, 17, 11, 3, 4, 19, 15, 18, 5, 0, 1, 8, 13, 9, 7, 12, 2]  makespan =  1428
FBA_swap_sol:  [13, 14, 10, 16, 17, 11, 3, 4, 19, 15, 18, 5, 0, 1, 8, 6, 9, 7, 12, 2]  makespan =  1424
FBA_swap_sol:  [13, 14, 10, 16, 7, 11, 3, 4, 19, 15, 18, 5, 0, 1, 8, 6, 9, 17, 12, 2]  makespan =  1420
FBA_swap_sol:  [13, 14, 10, 16, 7, 17, 3, 4, 19,

In [124]:
print('Results of PRSKE:')
print(f'First sequence: {initialSolution} with a makespan of {makespan}.')
print('\nResults of RS:')
print(f'Best solution: {rs_solution} with a makespan of {calculate_makespan(benchmark, rs_solution)}')
print('\nResults of RS_FBA:')
print(f'Best solution: {rs_fba_solution} with a makespan of {calculate_makespan(benchmark, rs_fba_solution)}.')
print(f'Best global solution: {best_global_found} with a makespan of {calculate_makespan(benchmark, best_global_found)}.')

Results of PRSKE:
First sequence: [3, 17, 10, 1, 9, 11, 6, 4, 19, 15, 18, 5, 0, 12, 8, 13, 14, 7, 16, 2] with a makespan of 1593.

Results of RS:
Best solution: [8, 16, 14, 2, 3, 1, 13, 4, 12, 6, 10, 7, 0, 18, 15, 5, 17, 11, 9, 19] with a makespan of 1297

Results of RS_FBA:
Best solution: [12, 14, 10, 16, 13, 17, 3, 4, 19, 15, 18, 5, 0, 1, 8, 9, 6, 11, 7, 2] with a makespan of 1343.
Best global solution: [12, 14, 10, 16, 13, 17, 3, 4, 19, 15, 18, 5, 0, 1, 8, 9, 6, 11, 7, 2] with a makespan of 1343.
